# Tutorial #6: Dropout and BatchNorm

### 6.1: Dropout

Dropout is a method of regularizing networks. Dropout is an extremely simple: during training, multiply the input to each unit by either 0 or 1, multiplying by 1 with probability $p$ and by 0 with probability $(1-p)$. Typically, $p=0.5$ is chosen for hidden units, as it empirically tends to work well in applications.

During testing, none of the units are multiplied by zero. Empiricially, it has been shown that dropout has better performance if the expected value of the output of each hidden unit matches the distribution seen during training. Since $1/p$ times more hidden units are included in each linear transformation during testing compared to training, the expected value of each output unit will be multiplied by $1/p$. To counteract this multiplication in expected value, the weights of the linear transformation are simply multiplied by $1/p$ during testing.

For a layer within a neural network with input $\boldsymbol{x}$, non-linear activation function $g(\boldsymbol{z})$, and linear transformation matrix $\boldsymbol{W}$, and bias $\boldsymbol{b}$, the output $\boldsymbol{h}$ of the hidden layer is given by $$\boldsymbol{h} = g(\boldsymbol{W} \boldsymbol{x} + \boldsymbol{b}).$$ Dropout performs elementwise multiplication of $\boldsymbol{x}$ with a binary mask $\boldsymbol{r}$ where each element of $\boldsymbol{r}$ is drawn from a Bernoulli distribution with probability $p$. During training, the dropout layer becomes $$r_j \sim \textnormal{Bernoulli}(p)$$ $$ \boldsymbol{\tilde{x}} = \boldsymbol{r} * \boldsymbol{x}$$ $$\boldsymbol{h} = g(\boldsymbol{W} \boldsymbol{\tilde{x}} + \boldsymbol{b}).$$ During testing, the layer now becomes $$\boldsymbol{h} = g(\frac{1}{p}\boldsymbol{W} \boldsymbol{x} + \boldsymbol{b}).$$

Dropout tends to take a different form for convolutional networks, recurrent networks, and attention layers than in MLPs. However, the basic idea of probabalistically setting hidden units to zero still remains.

Good resources for understanding dropout include the original 2014 paper by [Srivastava et al.](TODO), section 7.12 of the [Deep Learning Book](TODO), and [this blog post](https://medium.com/biased-algorithms/the-role-of-dropout-in-neural-networks-fffbaa77eee7#:~:text=Dropout%20has%20been%20successfully%20integrated,layers%20after%20the%20convolutional%20blocks.).

### 6.1.1: Dropout in `flax`

Dropout in `flax` is extremely easy to implement. We simply create a `nnx.Dropout(p, rngs)` layer, and call it to mask the input to the each layer. 

In [1]:
import flax
import jax
from flax import nnx
from jax import numpy as jnp

In [2]:
class DropoutTestMLP(nnx.Module):
    def __init__(self, rngs: nnx.Rngs):
        self.linear1 = nnx.Linear(4, 16, rngs=rngs)
        self.linear2 = nnx.Linear(16, 10, rngs=rngs)
        self.dropout = nnx.Dropout(0.5, rngs = rngs)

    def __call__(self, x):
        x = nnx.relu(self.linear1(x))
        return self.linear2(self.dropout(x))

We have to call `model.train()` to set the dropout layers to mask the inputs, and `model.eval()` to multiply the weights by $1/p$ for testing. 

In [3]:
model = DropoutTestMLP(nnx.Rngs(0))

model.train()

test_input = jax.random.normal(jax.random.PRNGKey(0), (4,))
print("First train dropout, mask chosen randomly")
print(model(test_input))
print("Second train dropout, mask chosen randomly with different RNG key")
print(model(test_input))

model.eval()

print("First eval dropout, no mask")
print(model(test_input))
print("Second eval dropout, deterministic output same as first eval")
print(model(test_input))

First train dropout, mask chosen randomly
[-0.24484156  1.3231333  -3.9299839  -3.0473146   3.225388    0.22339313
  0.5151764  -1.9571856   0.47121215  0.05025537]
Second train dropout, mask chosen randomly with different RNG key
[-1.6144576   0.61325073 -3.1674788  -2.2446618   2.230607   -0.08878337
  1.1725848   0.04968039  0.73547506  2.1024349 ]
First eval dropout, no mask
[-0.87284786  0.84970397 -1.2577161  -0.75616425  0.4537591  -0.6594648
  0.9112347   0.19184317  0.8342736   1.534824  ]
Second eval dropout, deterministic output same as first eval
[-0.87284786  0.84970397 -1.2577161  -0.75616425  0.4537591  -0.6594648
  0.9112347   0.19184317  0.8342736   1.534824  ]


When using Dropout layers, we have to remember to call `model.train()` before training and `model.eval()` before testing.

### 6.2: BatchNorm

BatchNorm is a way of improving the training process of neural networks with gradient descent. BatchNorm can be used simultaneously with Dropout, or it can be used without Dropout. 

BatchNorm is performed in two steps. 

First, for each activation, normalize the activation so that it has mean zero and variance 1. Instead of calculating the mean and variance over the entire dataset, during training BatchNorm uses the statistics from the current batch to estimate a mean and variance. So, for the $k$th activation value $x^{k}$ in a BatchNorm layer, during training the activation is normalized with the equation $$\hat{x}^{k} = \frac{x^k - \mathbb{E}[x^k]}{\sqrt{\textnormal{Var}[x^k]}}$$  where the expectation and variance are calculated using $$\mathbb{E}[x^k] = \frac{1}{m}\sum_{i=1}^n x^k_i$$ $$\textnormal{Var}[x^k] = \frac{1}{m-1}\sum_{i=1}^m (x^k_i - \mathbb{E}[x^k])^2 + \delta$$ where $\delta$ is a small number to prevent numerical underflow are calculated by summing over the batch. During testing, the means and variances are calculated using stored running averages from the training $$\mathbb{E}[x^k]_{\textnormal{running}} = \alpha \mathbb{E}[x^k]_{\textnormal{running}} + (1-\alpha) \mathbb{E}[x^k]_{\textnormal{batch}}$$ $$\sqrt{\textnormal{Var}[x^k]_{\textnormal{running}}} = \alpha \sqrt{\textnormal{Var}[x^k]_{\textnormal{running}}} + (1-\alpha) \sqrt{\textnormal{Var}[x^k]_{\textnormal{batch}}}$$ where $\alpha$ is a momentum term close to 1.

Second, each normalized activation $\hat{x}^k$ is then transformed by two trainable parameters, $\gamma^k$ and $\beta^k$. The final transformed activation value $h^k$ is given by $$h^k = \gamma^k \hat{x}^k + \beta^k.$$ This second transformation ensures that BatchNorm can, in practice, learn the identity transformation and thus does not reduce the functional approximation capability of the neural network.

Each transformation is differentiable, allowing BatchNorm layers to be incorporated into a gradient descent optimization algorithm.

One way of intuitively understanding the purpose of the first transformation is in terms of preventing activation function saturation. Recall that the sigmoid activation function $1/(1+e^{-x})$ has a linear regime near $x=0$ where the gradient is close to $1$, but a saturated regime at large $|x|$ where the gradient is close to zero. If the magnitude of the activation value is very large, the magnitude of the gradient will be very small and gradient descent will take a long time to converge. By normalizing the activations to have mean 0 and variance 1, fewer units will be in the saturated regime and the network can train faster.

The purpose of the second transformation is not immediately clear. After all, what is the purpose of multiplying by a variance and adding a mean right after subtracting off the mean and normalizing the variance? The explanation can be found by understanding the training dynamics of stacked transformations. Consider, for example, a linear network $$y = w_1 w_2 \dots w_n x.$$ Because $x$ is multiplied only by linear matrices, $y$ can only represent linear transformations of $x$. However, when this network is trained during gradient descent, each of the weights are updated according to the gradient times the learning rate. As a result, the activation means can undergo large changes even after a small gradient update, depending on the values of $w_i$ in the below layers. However, when the mean of each hidden unit is normalized using BatchNorm, the mean of each activation depends only on the parameter $\beta^k$, and not on a complicated interaction between the gradient descent step size and the layer weights. As a result, the network activation means don't change dramatically in response to each gradient descent step size. 

In practice, networks trained with BatchNorm can use larger learning rates than methods without BatchNorm. While training speed is not proportional to the learning rate used, empirically these larger learning rates lead to faster training speed. Furthermore, Batchnorm-optimized networks tend to have better performance than methods without using BatchNorm. 

Good introductory resources on BatchNorm include section 8.7.1 of the [Deep Learning Book](https://www.deeplearningbook.org/) and the original [BatchNorm paper](https://arxiv.org/abs/1502.03167). 

### 6.2.1: BatchNorm in Flax